# 📊 Hierarchical Sampling & Experiment Management

## Advanced SeedHash Tutorial #2

This notebook covers:
- `SeedExperimentManager` for systematic experimentation
- Hierarchical seed generation (master → seeds → sub-seeds)
- 4 sampling methods: simple, stratified, cluster, systematic
- ML experiment tracking with metrics
- DataFrame export and analysis

**Prerequisites**: Complete Tutorial #1 first

---

## Installation

If you haven't installed seedhash yet, run:

In [ ]:
# Install seedhash with all dependencies (uncomment if needed)
# !pip install "git+https://github.com/melhzy/seedhash.git#egg=seedhash[all]&subdirectory=Python"

In [1]:
# Setup and imports
import sys
sys.path.insert(0, '../Python')

import numpy as np
import pandas as pd
from seedhash import SeedExperimentManager

print("✅ All imports successful!")

✅ All imports successful!


## 1. Introduction to SeedExperimentManager 🎯

The `SeedExperimentManager` manages multiple experiments with hierarchical seeds.

In [2]:
# Create an experiment manager
manager = SeedExperimentManager("my_ml_project")

print(f"Project: {manager.experiment_name}")
print(f"Master seed: {manager.master_seed}")
print(f"Experiments tracked: {len(manager.results)}")

Project: my_ml_project
Master seed: 3002848522
Experiments tracked: 0


## 2. Hierarchical Seed Generation 🌳

Generate hierarchy: master → seeds → sub-seeds

In [3]:
# Generate hierarchical seeds
hierarchy = manager.generate_seed_hierarchy(
    n_seeds=3,
    n_sub_seeds=2,
    max_depth=2
)

print(f"Master seed: {hierarchy[0][0]}\n")
print(f"Level 0 (master): {hierarchy[0]}")
print(f"Level 1 (seeds): {hierarchy[1]}")
print(f"Level 2 (sub-seeds): {hierarchy[2]}")

print(f"\nTotal: 1 master → {len(hierarchy[1])} seeds → {len(hierarchy[2])} sub-seeds")

Master seed: 3002848522

Level 0 (master): [3002848522]
Level 1 (seeds): [5644790, 1216290993, 1629135287]
Level 2 (sub-seeds): [1240067319, 370689488, 732425434, 2111485999, 2141033051, 862800348]

Total: 1 master → 3 seeds → 6 sub-seeds


## 3. Simple Random Sampling 🎲

In [4]:
# Simple random sampling using SeedSampler
from seedhash import SeedSampler

sampler = SeedSampler(master_seed=12345)
samples = sampler.simple_random_sampling(
    n_samples=100,
    seed_range=(0, 1000)
)

print(f"Sample size: {len(samples)}")
print(f"First 10: {sorted(samples)[:10]}")
print(f"Range: {min(samples)} to {max(samples)}")

Sample size: 100
First 10: [3, 6, 10, 24, 27, 41, 79, 93, 101, 108]
Range: 3 to 990


## 4. Stratified Random Sampling 📊

In [5]:
# Stratified sampling ensures balanced coverage
sampler = SeedSampler(master_seed=12345)
samples = sampler.stratified_random_sampling(
    n_samples=100,
    seed_range=(0, 1000),
    n_strata=10
)

print(f"Stratified sample size: {len(samples)}")
print(f"Expected per stratum: ~{100//10}")
print(f"Range: {min(samples)} to {max(samples)}")
print(f"\nDistribution check (first 30):")
print(sorted(samples)[:30])

Stratified sample size: 100
Expected per stratum: ~10
Range: 1 to 991

Distribution check (first 30):
[1, 20, 24, 34, 38, 47, 53, 55, 72, 93, 115, 122, 123, 133, 147, 155, 170, 171, 178, 180, 211, 221, 245, 252, 264, 267, 274, 293, 294, 294]


## 5. ML Experiment Tracking 📈

In [6]:
# Track ML experiments
tracker = SeedExperimentManager("ml_tracking_demo")

hierarchy = tracker.generate_seed_hierarchy(n_seeds=5, n_sub_seeds=2, max_depth=2)

# Simulate experiments
for seed in hierarchy[1][:3]:
    rmse = 5.0 + np.random.rand()
    r2 = 0.95 + np.random.rand() * 0.04
    
    tracker.add_experiment_result(
        seed=seed,
        ml_task="regression",
        metrics={"rmse": rmse, "r2": r2, "mae": rmse * 0.8},
        sampling_method="simple",
        metadata={"model": "linear_regression", "n_samples": 100}
    )
    print(f"Tracked seed {seed}: RMSE={rmse:.3f}, R²={r2:.3f}")

print(f"\n✓ Tracked {len(tracker.results)} experiments")

Tracked seed 1709912838: RMSE=5.929, R²=0.955
Tracked seed 374373719: RMSE=5.930, R²=0.961
Tracked seed 1166838137: RMSE=5.793, R²=0.962

✓ Tracked 3 experiments


## 6. DataFrame Export & Analysis 📊

In [7]:
# Export to DataFrame
df = tracker.get_results_dataframe()

print("DataFrame Preview:")
print(df.head())

print(f"\nShape: {df.shape}")
print(f"\nMetric Statistics:")
print(df[['metric_rmse', 'metric_r2', 'metric_mae']].describe())

# Export to files
df.to_csv('experiment_results.csv', index=False)
print("\n✓ Exported to CSV!")

DataFrame Preview:
                                experiment_id  seed_level  master_seed  \
0  ml_tracking_demo_regression_seed1709912838           1   3662338350   
1   ml_tracking_demo_regression_seed374373719           1   3662338350   
2  ml_tracking_demo_regression_seed1166838137           1   3662338350   

         seed sub_seed  current_seed sampling_method     ml_task  metric_mae  \
0  1709912838     None    1709912838          simple  regression    4.742934   
1   374373719     None     374373719          simple  regression    4.744083   
2  1166838137     None    1166838137          simple  regression    4.634143   

   metric_r2  metric_rmse         meta_model  meta_n_samples  \
0   0.955179     5.928668  linear_regression             100   
1   0.960708     5.930104  linear_regression             100   
2   0.961860     5.792678  linear_regression             100   

                    timestamp  
0  2025-11-03T00:41:26.954905  
1  2025-11-03T00:41:26.954905  
2  2025-11

## 7. Complete Example: Multi-Method Study 🔬

In [8]:
# Compare all sampling methods
study = SeedExperimentManager("complete_study")

for method in ["simple", "stratified", "cluster", "systematic"]:
    hierarchy = study.generate_seed_hierarchy(
        n_seeds=3,
        n_sub_seeds=2,
        max_depth=2,
        sampling_method=method
    )
    
    for seed in hierarchy[1]:
        accuracy = 0.80 + np.random.rand() * 0.15
        f1 = accuracy * (0.95 + np.random.rand() * 0.05)
        
        study.add_experiment_result(
            seed=seed,
            ml_task="classification",
            metrics={"accuracy": accuracy, "f1": f1},
            sampling_method=method
        )
    
    print(f"✓ Completed {method} sampling")

df = study.get_results_dataframe()
print(f"\nTotal experiments: {len(df)}")
print("\nAccuracy by method:")
print(df.groupby('sampling_method')['metric_accuracy'].agg(['mean', 'std']).round(3))

✓ Completed simple sampling
✓ Completed stratified sampling
✓ Completed cluster sampling
✓ Completed systematic sampling

Total experiments: 12

Accuracy by method:
                  mean    std
sampling_method              
cluster          0.922  0.014
simple           0.897  0.065
stratified       0.888  0.061
systematic       0.825  0.025


## Summary 🎉

You learned:
- ✅ SeedExperimentManager for systematic experiments
- ✅ Hierarchical seed generation
- ✅ 4 sampling methods
- ✅ ML experiment tracking
- ✅ DataFrame export

**Next**: Try Tutorial #3 for advanced ML paradigms!